In [2]:
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tkinter import Tk
from tkinter.filedialog import askdirectory

import pathlib



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

batch_size = 32
image_size = 224, 224


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = '/content/drive/MyDrive/resized_images',
    validation_split=0.2,
    subset='training',
    seed=1337,
    batch_size=batch_size,
    image_size=image_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = '/content/drive/MyDrive/resized_images',
    validation_split=0.2,
    subset='validation',
    seed=1337,
    batch_size=batch_size,
    image_size=image_size
)




Found 197 files belonging to 2 classes.
Using 158 files for training.
Found 197 files belonging to 2 classes.
Using 39 files for validation.


In [ ]:
#Data augmentation (why should I use it?)
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


In [ ]:
#xception model
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    #rescale
    x = layers.experimental.preprocessing.Rescaling(1./255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual]) 
        previous_block_activation = x 

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
5/5 [==============================] - 32s 2s/step - loss: 0.7972 - accuracy: 0.6646 - val_loss: 0.6934 - val_accuracy: 0.4872


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
5/5 [==============================] - 2s 391ms/step - loss: 0.5014 - accuracy: 0.8038 - val_loss: 0.6938 - val_accuracy: 0.4872
Epoch 3/50
5/5 [==============================] - 2s 391ms/step - loss: 0.4222 - accuracy: 0.7722 - val_loss: 0.6937 - val_accuracy: 0.4872
Epoch 4/50
5/5 [==============================] - 2s 385ms/step - loss: 0.3710 - accuracy: 0.8228 - val_loss: 0.6947 - val_accuracy: 0.4872
Epoch 5/50
5/5 [==============================] - 2s 402ms/step - loss: 0.3249 - accuracy: 0.8291 - val_loss: 0.6955 - val_accuracy: 0.4872
Epoch 6/50
5/5 [==============================] - 2s 407ms/step - loss: 0.3045 - accuracy: 0.8924 - val_loss: 0.6969 - val_accuracy: 0.4872
Epoch 7/50
5/5 [==============================] - 2s 397ms/step - loss: 0.2630 - accuracy: 0.8797 - val_loss: 0.6990 - val_accuracy: 0.4872
Epoch 8/50
5/5 [==============================] - 2s 391ms/step - loss: 0.4016 - accuracy: 0.8544 - val_loss: 0.6989 - val_accuracy: 0.4872
Epoch 9/50
5/5 [====

In [3]:
img = keras.preprocessing.image.load_img(
    "/content/drive/MyDrive/2588705_1_articlelarge_basmati_rice_iStock-186835225.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent rice and %.2f percent not rice."
    % (100 * (1 - score), 100 * score)
)

TclError: ignored